<a href="https://colab.research.google.com/github/ishantmr/learning/blob/main/pytorch_tng_ppln_nn_builtin_ds_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaner version where we are importing NN Module and using its built in functions

# Added Dataset and Dataloader

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [7]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [12]:
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [13]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Defining the model

In [15]:
class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()


  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out

Important Parametes

In [16]:
learning_rate = 0.1
epochs = 25

In [17]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# optimization
# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

loss_function = nn.BCELoss() #Binary Cross Entropy Loss built in function in NN Module



# Training pipeline

In [19]:
# define loop
for epoch in range(epochs):

  for batch_features, batch_lables in train_loader:
    # forward pass
    y_pred = model(batch_features.float()) # Convert input tensor to float

    # loss calculate
    loss = loss_function(y_pred, batch_lables.view(-1,1).float())

    #clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    #paramter update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.166523739695549
Epoch: 2, Loss: 0.11959286034107208
Epoch: 3, Loss: 0.2619583010673523
Epoch: 4, Loss: 0.08603943139314651
Epoch: 5, Loss: 0.08672357350587845
Epoch: 6, Loss: 0.09753703325986862
Epoch: 7, Loss: 0.06502845138311386
Epoch: 8, Loss: 0.02674304135143757
Epoch: 9, Loss: 0.39634376764297485
Epoch: 10, Loss: 0.10748909413814545
Epoch: 11, Loss: 0.007072020322084427
Epoch: 12, Loss: 0.061186403036117554
Epoch: 13, Loss: 0.013474118895828724
Epoch: 14, Loss: 0.034173790365457535
Epoch: 15, Loss: 0.06743282079696655
Epoch: 16, Loss: 0.04162944108247757
Epoch: 17, Loss: 0.07161644846200943
Epoch: 18, Loss: 0.012603670358657837
Epoch: 19, Loss: 0.0787634328007698
Epoch: 20, Loss: 0.015239746309816837
Epoch: 21, Loss: 0.060563087463378906
Epoch: 22, Loss: 0.04523582011461258
Epoch: 23, Loss: 0.06634601205587387
Epoch: 24, Loss: 0.32414960861206055
Epoch: 25, Loss: 0.26564812660217285


In [21]:
# model evaluation using test loader

model.eval()
accuracy_list = []

with torch.no_grad():
  for batch_features, batch_lables in test_loader:
    #Forward Pass
    y_pred = model(batch_features.float())
    y_pred = (y_pred > 0.8).float()

    # Calculating batch accuracy
    batch_accuracy = (y_pred.view(-1) == batch_lables).float().mean()
    accuracy_list.append(batch_accuracy)

# Calculating overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')

Accuracy: 0.9549
